In [4]:
import numpy as np

In [10]:
import sklearn

__Простая эвристика__: давайте выдавать такой же ответ, какой является правильным для объекта обучающей выборки, наиболее похожего на данный (это будет наша _модель ответа_). 
Такой классификатор называется __Метод ближайшего соседа__.

В его основе лежит _предположение_ о том, что объекты одного класса лежат близко друг к другу в пространстве признаков (_гипотеза компактности_).

Стоит обратить внимание, что это чуть ли не единственный метод машинного обучения, в котором процедура _обучения_ состоит в _запоминании_ выборки (не нужно ничего оптимизировать).

Например, легко понять, что для двумерного случая классификация всей плоскости задается разбиением ее на многоугольники, ограниченные серединными перпендикулярами к отрезкам, попарно соединяющим объекты обучающей выборки.
Это показано на левой картинке (здесь выборка состоит из 8 объектов, целевой признак --- цвет точки: синий или зеленый, каждый объект представлен двумя признаками-координатами на вещественной плоскости).

![](https://shapeofdata.files.wordpress.com/2013/03/knn1.png)

Здесь кроется и вся опасность МБС --- его склонность к переобучению: для любого шумового объекта (например, правой нижней синей точки) будет создаваться отдельная область, то есть алгоритм крайне чувствителен к шуму.

Давайте немного усложним классификатор: вместо одного соседа будем рассматривать k наиболее похожих на данный объект, и выбирать тот класс, который наиболее часто встречается среди k соседей (новая _модель_). Получится классификатор, более устойчивый к переобучению. Этот метод называется __метод k ближайших соседей, или KNN__. Например, на правой картинке показана разделяющая поверхность при k=3, и там уже синий объект признается шумовым.

<img src="http://datasciencetoday.net/images/demo/k_NN.png">

### Обсуждение kNN

Для ряда задач (например, для распознавания цифр или классификации текстов, представленных в виде <<мешка слов>>) kNN дает хороший результат. Ключевой вопрос состоит в __выборе метрики для сравнения объектов__. Для вещественных объектов $x = (x_1, \dots, x_d)$ и $y=(y_1, \dots, y_d)$ используют:
1. расстояние Минковского с показателем $p$: $\rho(x, y) = \biggl(\sum_{i=1}^d (x_i - y_i)^p\biggr)^{\frac 1 p}$; при $p=2$ это привычное _евклидово расстояние_.
1. косинусную метрику:  $\rho(x, y) = \frac {<x, y>} {\sqrt{<x, x><y, y>}}$, $<. ,.>$ --- скалярное произведение: $<x, y> = \sum_{i=1}^d x_i y_i$. Это расстояние равно косинусу угла между векторами $x$ и $y$ в евклидовом пространстве; хорошо работает для текстов.

Для бинарных векторов (каждый компонент есть 0 или 1):
1. расстояние Хэмминга --- число несовпадающих разрядов. 
1. расстояние Жаккарда --- число единиц в поэлементной конъюнкции, деленное на число элемент в поэлементной дизъюнкции.

Последнее есть интерпретация расстояния Жаккарда для множеств A и B:
1. $\rho(A, B) = \frac {|A \cap B|}{|A \cup B|}$

(Подробнее о метриках здесь: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics)

Если в задаче известны признаки разной природы, можно составлять комбинированную метрику. 

Более того, можно вводить веса признаков $w = (w_1, \dots, w_d)$ и модифицировать метрики, например:
$\rho(x, y) = \sqrt{\sum_{i=1}^d w_i(x_i - y_i)^2}$.

Последний прием важен на _ненормированных_ выборках (если в сумме одно слагаемое имеет порядок 1000, а другое 0.1, то второе не будет иметь влияния на величину расстояния). Тогда в качестве весов можно использовать обратные к средним или к максимальным значениям признаков.

Веса можно применять и к объектам, тогда объекты, находящиеся на расстоянии, более близком к новой точке, будут иметь большее влияние.

### kNN в sklearn

Интерфейс описан в [документации](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [8]:
# импортируем классификатор
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)

In [57]:
# загружаем данные --- изображения цифр
from sklearn.datasets import load_digits
data = load_digits()

In [58]:
X = data.images
y = data.target

In [59]:
X.shape

(1797L, 8L, 8L)

In [60]:
X = X.reshape(X.shape[0], -1) # вытягиваем квадратное изображение в вектор, чтобы получить матрицу объекты-признаки

In [52]:
from sklearn.utils import shuffle

In [53]:
X, y = shuffle(X, y)

In [54]:
X.shape, y.shape # проверяем, что все хорошо перемешалось

((1797L, 64L), (1797L,))

In [55]:
y[:10] # теперь в случайном порядке

array([1, 4, 8, 6, 5, 2, 5, 6, 5, 6])

In [74]:
X_train, y_train = X[:700, :], y[:700]
X_val, y_val = X[700:1300, :], y[700:1300] #validation
X_test, y_test = X[1300:, :], y[1300:]

In [79]:
# Обучаем классификатор и делаем предсказания
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)

In [80]:
# Вычисляем простейшую метрику качества алгоритма --- долю правильных ответов
print("Accuracy is", np.mean(y_test==y_predicted))

Accuracy is 0.955734406439


Учитывая, что у нас 10 классов, и вероятность случайно вытащить два одинаковых маленькая, точность 0.956 --- очень хороший результат!

In [84]:
# Подбор k на валидационной выборке:
for k in range(1, 20):
    y_predicted = KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train).predict(X_val)
    print("k =", k, "; accuracy =", np.mean(y_predicted==y_val))

k = 1 ; accuracy = 0.958333333333
k = 2 ; accuracy = 0.956666666667
k = 3 ; accuracy = 0.956666666667
k = 4 ; accuracy = 0.956666666667
k = 5 ; accuracy = 0.958333333333
k = 6 ; accuracy = 0.958333333333
k = 7 ; accuracy = 0.963333333333
k = 8 ; accuracy = 0.965
k = 9 ; accuracy = 0.965
k = 10 ; accuracy = 0.966666666667
k = 11 ; accuracy = 0.958333333333
k = 12 ; accuracy = 0.961666666667
k = 13 ; accuracy = 0.96
k = 14 ; accuracy = 0.956666666667
k = 15 ; accuracy = 0.951666666667
k = 16 ; accuracy = 0.946666666667
k = 17 ; accuracy = 0.94
k = 18 ; accuracy = 0.94
k = 19 ; accuracy = 0.94


Наилучший результат при k=10

**Плюсы и минусы метода ближайших соседей**

Плюсы:


1. Простая реализация;

2. Неплохо изучен теоретически;

3. Как правило, метод хорош для первого решения задачи, причем не только классификации или регрессии, но и, например, рекомендации;

4. Можно адаптировать под нужную задачу выбором метрики или ядра (в двух словах: ядро может задавать операцию сходства для сложных объектов типа графов, а сам подход kNN остается тем же);

5. Неплохая интерпретация, можно объяснить, почему тестовый пример был классифицирован именно так. Хотя этот аргумент можно атаковать: если число соседей большое, то интерпретация ухудшается (условно: "мы не дали ему кредит, потому что он похож на 350 клиентов, из которых 70 – плохие, что на 12% больше, чем в среднем по выборке").

Минусы:


1. Метод считается быстрым в сравнении, например, с композициями алгоритмов, но в реальных задачах, как правило, число соседей, используемых для классификации, будет большим (100-150), и в таком случае алгоритм будет работать не так быстро, как дерево решений;

2. Если в наборе данных много признаков, то трудно подобрать подходящие веса и определить, какие признаки не важны для классификации/регрессии;

3. Зависимость от выбранной метрики расстояния между примерами. Выбор по умолчанию евклидового расстояния чаще всего ничем не обоснован. Можно отыскать хорошее решение перебором параметров, но для большого набора данных это отнимает много времени;

4. Нет теоретических оснований выбора определенного числа соседей — только перебор (впрочем, чаще всего это верно для всех гиперпараметров всех моделей). В случае малого числа соседей метод чувствителен к выбросам, то есть склонен переобучаться;

5. Как правило, плохо работает, когда признаков много, из-за "проклятия размерности".